In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import pickle

In [3]:

df = pd.read_csv('/kaggle/input/credit-scores-csv/credit_scores.csv')
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Score,Count_Auto Loan,Count_Credit-Builder Loan,Count_Personal Loan,Count_Home Equity Loan,Count_Not Specified,Count_Mortgage Loan,Count_Student Loan,Count_Debt Consolidation Loan,Count_Payday Loan
0,0x1608,CUS_0xd40,July,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3.0,...,Good,1.0,1.0,1.0,1.0,0.0,0,0,0,0
1,0x160f,CUS_0x21b1,February,Rick Rothackerj,28.0,004-07-5839,Teacher,34847.84,3037.986667,2.0,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
2,0x1612,CUS_0x21b1,May,Rick Rothackerj,28.0,004-07-5839,Teacher,34847.84,3037.986667,2.0,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
3,0x1613,CUS_0x21b1,June,Rick Rothackerj,28.0,004-07-5839,Teacher,34847.84,3037.986667,2.0,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
4,0x1615,CUS_0x21b1,August,Rick Rothackerj,28.0,004-07-5839,Teacher,34847.84,3037.986667,2.0,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33397,0x25fda,CUS_0x8600,January,Sarah McBridec,28.0,031-35-0942,Architect,20002.88,1929.906667,10.0,...,Poor,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33398,0x25fdb,CUS_0x8600,February,Sarah McBridec,28.0,031-35-0942,Architect,20002.88,1929.906667,10.0,...,Poor,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33399,0x25fe0,CUS_0x8600,July,Sarah McBridec,28.0,031-35-0942,Architect,20002.88,1929.906667,10.0,...,Standard,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33400,0x25fe1,CUS_0x8600,August,Sarah McBridec,29.0,031-35-0942,Architect,20002.88,1929.906667,10.0,...,Standard,1.0,0.0,1.0,0.0,0.0,1,2,0,0


In [4]:

df = df.drop(columns=["Name", "SSN", "ID", "Customer_ID"])
X = df.drop('Credit_Score', axis=1)
y = df['Credit_Score']
df

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Score,Count_Auto Loan,Count_Credit-Builder Loan,Count_Personal Loan,Count_Home Equity Loan,Count_Not Specified,Count_Mortgage Loan,Count_Student Loan,Count_Debt Consolidation Loan,Count_Payday Loan
0,July,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,3.0,8,...,Good,1.0,1.0,1.0,1.0,0.0,0,0,0,0
1,February,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,7.0,1,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
2,May,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,3.0,1,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
3,June,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,3.0,0,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
4,August,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,3.0,4,...,Good,0.0,1.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33397,January,28.0,Architect,20002.88,1929.906667,10.0,8.0,29.0,35.0,26,...,Poor,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33398,February,28.0,Architect,20002.88,1929.906667,10.0,8.0,29.0,33.0,25,...,Poor,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33399,July,28.0,Architect,20002.88,1929.906667,10.0,8.0,29.0,33.0,26,...,Standard,1.0,0.0,1.0,0.0,0.0,1,2,0,0
33400,August,29.0,Architect,20002.88,1929.906667,10.0,8.0,29.0,33.0,25,...,Standard,1.0,0.0,1.0,0.0,0.0,1,2,0,0


In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
# Identifying numerical and categorical columns
num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

# Handling missing values and scale numerical columns
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Handling missing values and convert categorical columns into numerical columns
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())])

# Combining preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)])

In [7]:
# Developing a Support Vector Machine model
model = SVC()

# Create a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

In [ ]:
# Fine-tunning hyperparameters
# Defining the parameter grid
param_grid = {'classifier__kernel': ['rbf', 'linear'], 'classifier__C': [0.01, 10, 20]}

# Using randomized search with a specified number of iterations
random_search = RandomizedSearchCV(clf, param_grid, cv=5, n_iter=4, random_state=1)
random_search.fit(X_train, y_train)

# Reporting the accuracy of the best model
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

In [ ]:
# Retraining the best model using the whole dataset
best_model = random_search.best_estimator_
best_model.fit(X, y)

In [ ]:
# Calculating the score of the best model on training data
train_score = best_model.score(X_train, y_train)
print("Training score: ", train_score)

# Calculating the score of the best model on test data
test_score = best_model.score(X_test, y_test)
print("Test score: ", test_score)

In [ ]:
# Saving the best model as a file
filename = 'credit_scores_MLproj.pkl'

# Saving the model to disk
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)